In [1]:
#   載入套件
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
from datetime import datetime

In [2]:
# 目標網址 (已爬取當日的最新文章編碼847039)
# https://money.udn.com/money/story/5612/8470390?from=edn_subcatelist_cate
news_number = 8470390


# 設立空的list
output = []
output.clear()

# 計算總嘗試數 與 失敗數
count = 0
null = 0
# 取得50篇文章
while (len(output)<60):
    
    news_number = news_number - count
    url = f"https://money.udn.com/money/story/5612/{news_number}?from=edn_subcatelist_cate"
    try:
        # 取得網頁html 並解析
        res = requests.get(url)
        soup = bs(res.text, "lxml")
        # 取得標題
        title = soup.select_one('h1').get_text().strip()

        # 取得內文
        results = soup.select("section.article-body__editor>p")

        #去除圖片文字說明
        for result in results:
            for del_figure in result.select('figure'):
                del_figure.decompose()

        # 去除空字串 以延伸閱讀當作文章終點
        content=""
        for i in results:
            i = i.get_text().strip()
            if i == '':
                continue
            content = content + i
            if i == '延伸閱讀':
                break

        # 取得文章內文發佈時間 html中第一個時間
        date = soup.select_one("time").get_text().strip()

        # 將時間轉換西元年月日時分秒形式
        date = datetime.strptime(date, "%Y/%m/%d %H:%M:%S")
        date = date.strftime("%Y%m%d%H%M")
        # 將文章內容以字典形式加入結果字串
        output.append({
                    "sources":"money-udn",
                    "title":title,
                    "url":url,
                    "time":date,
                    "results":content})
        count += 1
    except:
        count += 1
        null += 1
        continue
    
# 總共試了幾個文章代碼
print(count)
# 有多少無效文章代碼
print(null)

169
109


In [3]:
# 將結果轉為data frame
df = pd.DataFrame(output)
# 取得文章數量
print(len(df))
# 輸出為csv檔
df.to_csv('money_udn', index=True, encoding='utf-8-sig')

60
